In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import Bibliothèque
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from tqdm import tqdm
from rdkit.Chem import AllChem, DataStructs

In [4]:
#Importation des fichiers csv 
folder_path = "/kaggle/input/melting-point"
train_csv_path = '/kaggle/input/melting-point/train.csv'
train_csv = pd.read_csv(train_csv_path)
#print(train_csv.head())

test_csv_path = '/kaggle/input/melting-point/test.csv'
test_csv = pd.read_csv(test_csv_path)
print(test_csv.head())

     id               SMILES  Group 1  Group 2  Group 3  Group 4  Group 5  \
0  1022  CCOC(=O)c1ccc(O)cc1        1        1        0        0        0   
1  1146   CCCCCCc1ccc(O)cc1O        1        4        0        0        0   
2    79                ClCBr        0        0        0        0        0   
3  2279          C=CCCCCCCCC        1        7        0        0        1   
4  1342  Fc1ccc(cc1)C(F)(F)F        0        0        0        0        0   

   Group 6  Group 7  Group 8  ...  Group 415  Group 416  Group 417  Group 418  \
0        0        0        0  ...          0          0          0          0   
1        0        0        0  ...          0          0          0          0   
2        0        0        0  ...          0          0          0          0   
3        0        0        0  ...          0          0          0          0   
4        0        0        0  ...          0          0          0          0   

   Group 419  Group 420  Group 421  Group 422  Gro

In [5]:
#Chargement du Dataset complet
#Full dataset loading
train_data_set = train_csv

y = train_data_set['Tm']
list_columns = train_data_set.columns.tolist()
feature_names = list_columns[3:]

# print the list of columns in the dataset to find the name of the prediction target
#train_data_set.describe()

test_data_set = test_csv

In [6]:
#Retraitement des données sous la forme de 0 et 1 - Train
X_df = pd.DataFrame(train_data_set[feature_names])

#Créer les colonnes binaires
binary_feats = {f"has_{col}": (X_df[col] > 0).astype(int) for col in feature_names}

#Ratios 
ratio_feats = {f"{col}_ratio": X_df[col] / (X_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}

#Totaux globaux
extra_feats = {
    "total_groups": X_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_df[feature_names] > 0).sum(axis=1),
}

# 5. Fusionner toutes les nouvelles colonnes d’un coup
X_df = pd.concat([X_df, pd.DataFrame({**binary_feats, **ratio_feats, **extra_feats})], axis=1)

print("Shape après feature engineering:", X_df.shape)

Shape après feature engineering: (2662, 1274)


In [7]:
#Traitement des SMILES
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem

X_smiles = pd.DataFrame(train_data_set["SMILES"])

def compute_chem_features(df, smiles_col="smiles", add_fingerprint=True, fp_nbits=2048):
    """
    Prend un DataFrame avec une colonne de SMILES et renvoie un DataFrame de descripteurs chimiques.
    - df : DataFrame d'entrée (ne modifie pas df).
    - smiles_col : nom de la colonne SMILES.
    - add_fingerprint : bool, calcule fingerprint Morgan (binaire) si True.
    - fp_nbits : taille du vecteur fingerprint.
    """
    records = []
    invalid_idx = []
    
    # SMARTS motifs utiles (exemples)
    smarts = {
        "has_OH": "[OX2H]",              # alcool phénol etc. (H-bond donors)
        "has_carbonyl": "[CX3]=O",      # carbonyle
        "has_nitro": "[NX3](=O)=O",     # nitro
        "has_amine": "[NX3;!$(N-*=O)]", # amine primaire/secondaire/tertiaire (élimine nitro)
        "has_ester": "[$([CX3](=O)O)]",  # ester
        "has_thiol": "[SX2H]",           # thiol
    }
    smarts_compiled = {k: Chem.MolFromSmarts(v) for k, v in smarts.items()}
    
    for idx, smi in df[smiles_col].items():
        mol = None
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                # try sanitize false then sanitize
                mol = Chem.MolFromSmiles(smi, sanitize=False)
                if mol is not None:
                    Chem.SanitizeMol(mol)
        except Exception as e:
            mol = None
        
        if mol is None:
            invalid_idx.append(idx)
            # fill NaNs for invalid
            records.append({"index": idx, "valid_smiles": False})
            continue
        
        r = {"index": idx, "valid_smiles": True}
        
        # Atom counts (heavy atoms, heteroatoms, and halogens)
        atom_counts = {}
        for atom in mol.GetAtoms():
            sym = atom.GetSymbol()
            atom_counts[sym] = atom_counts.get(sym, 0) + 1
        # common atoms
        for at in ["C","H","O","N","F","Cl","Br","I","S","P"]:
            r[f"atom_count_{at}"] = atom_counts.get(at, 0)
        r["heavy_atom_count"] = rdMolDescriptors.CalcNumHeavyAtoms(mol)
        
        # Basic descriptors
        r["MolWt"] = Descriptors.ExactMolWt(mol)           # masse exacte
        r["MolLogP"] = Descriptors.MolLogP(mol)            # octanol-water partition
        r["MolMR"] = Descriptors.MolMR(mol)                # molar refractivity (approx)
        r["TPSA"] = rdMolDescriptors.CalcTPSA(mol)         # topological polar surface area
        r["NumHDonors"] = rdMolDescriptors.CalcNumHBD(mol)
        r["NumHAcceptors"] = rdMolDescriptors.CalcNumHBA(mol)
        r["NumRotatableBonds"] = rdMolDescriptors.CalcNumRotatableBonds(mol)
        r["NumRings"] = rdMolDescriptors.CalcNumRings(mol)
        r["NumAromaticRings"] = rdMolDescriptors.CalcNumAromaticRings(mol)
        r["FractionCSP3"] = Descriptors.FractionCSP3(mol)
        r["NumValenceElectrons"] = sum([a.GetTotalValence() for a in mol.GetAtoms()])
        r["FormalCharge"] = Chem.GetFormalCharge(mol)
        
        r["RingCount"] = Descriptors.RingCount(mol)
        r["HeavyAtomCount"] = Descriptors.HeavyAtomCount(mol)
        r["NumAliphaticRings"] = rdMolDescriptors.CalcNumAliphaticRings(mol)
        r["NumSaturatedRings"] = rdMolDescriptors.CalcNumSaturatedRings(mol)
        r["NumAliphaticCarbocycles"] = rdMolDescriptors.CalcNumAliphaticCarbocycles(mol)
        r["NumAromaticCarbocycles"] = rdMolDescriptors.CalcNumAromaticCarbocycles(mol)
        r["NumSaturatedCarbocycles"] = rdMolDescriptors.CalcNumSaturatedCarbocycles(mol)

        #Ajout de ratios supplémentaires
        r["PMolWtTPSA"] = r["MolWt"] * r["TPSA"]
        r["PMolLogPNumHDonors"] = r["MolLogP"] * r["NumHDonors"]
        r["PhbondMolWt"] = r["MolWt"]
        r["PNumAromRings"] = r["NumAromaticRings"] / (r["NumRings"]+1)
        
        # ring/aromaticity / complexity
        r["BertzCT"] = Descriptors.BertzCT(mol)             # complexité topologique
        # optionally: r["BalabanJ"] = Descriptors.J(mol)
        
        # SMARTS motif counts/presence
        for name, patt in smarts_compiled.items():
            try:
                matches = mol.GetSubstructMatches(patt)
                r[f"{name}_count"] = len(matches)
                r[f"{name}_present"] = int(len(matches) > 0)
            except Exception:
                r[f"{name}_count"] = 0
                r[f"{name}_present"] = 0
        
        # hydrogen bonding index (simple)
        r["hbond_index"] = r["NumHDonors"] * r["NumHAcceptors"]
        r["PhbondMolWt"] = r["hbond_index"] * r["MolWt"]
        # aromatic proportion
        r["aromatic_atom_fraction"] = sum(1 for a in mol.GetAtoms() if a.GetIsAromatic()) / max(1, mol.GetNumAtoms())
        
        # Morgan fingerprint (ECFP) as bit vector (optional)
        if add_fingerprint:
            try:
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=fp_nbits)
                arr = np.zeros((1,), dtype=np.int8)
                # convert to numpy array
                onbits = list(fp.GetOnBits())
                # We'll create sparse representation: store as list of 0/1 ints in columns fp_0..fp_nbits-1
                for i in range(fp_nbits):
                    r[f"fp_{i}"] = 1 if i in onbits else 0
            except Exception:
                # fallback: no fp
                for i in range(fp_nbits):
                    r[f"fp_{i}"] = 0
        
        records.append(r)
    
    feats = pd.DataFrame(records).set_index("index")
    # Convert types and fill NaN for invalid SMILES
    feats = feats.sort_index()
    feats = feats.fillna(np.nan)
    
    # Optionally drop fp columns if too heavy
    return feats, invalid_idx

# Utilisation :
feats_df, invalids = compute_chem_features(X_smiles, smiles_col="SMILES", add_fingerprint=False)
print("Invalid SMILES indices:", invalids)
X_df = pd.concat([X_df, pd.DataFrame({**feats_df})], axis=1)
print("Shape après feature engineering chimique:", X_df.shape)
#print(X_df.head())

Invalid SMILES indices: []
Shape après feature engineering chimique: (2662, 1324)


In [8]:
#Génération des Morgan Fingerprints

# Configuration
smiles_col="SMILES"   # nom de la colonne contenant les SMILES
n_bits = 512            # taille du vecteur (128, 256, 512 — 256 est un bon compromis)
radius = 2             # rayon (2 = bon point de départ)

def morgan_fingerprint(smi, n_bits=256, radius=2):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return np.zeros(n_bits, dtype=int)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# --- Application à ton dataset ---
print("🧬 Calcul des empreintes Morgan...")
fps = np.array([morgan_fingerprint(smi, n_bits, radius) for smi in tqdm(X_smiles[smiles_col])])

# --- Conversion en DataFrame ---
fp_cols = [f"morgan_{i}" for i in range(n_bits)]
df_fps = pd.DataFrame(fps, columns=fp_cols)

# --- Fusion avec ton DataFrame existant ---
X_df = pd.concat([X_df.reset_index(drop=True), df_fps], axis=1)

print(f"✅ Ajouté {n_bits} nouvelles colonnes de features Morgan.")
print(f"Nouvelle taille du DataFrame : {X_df.shape}")

🧬 Calcul des empreintes Morgan...


100%|██████████| 2662/2662 [00:00<00:00, 10270.77it/s]

✅ Ajouté 512 nouvelles colonnes de features Morgan.
Nouvelle taille du DataFrame : (2662, 1836)


In [9]:
#Retraitement du fichier Test
X_test_df = pd.DataFrame(test_data_set[feature_names])
X_test_smiles = pd.DataFrame(test_data_set["SMILES"])
#Créer les colonnes binaires
binary_feats_test = {f"has_{col}": (X_test_df[col] > 0).astype(int) for col in feature_names}
ratio_feats_test = {f"{col}_ratio": X_test_df[col] / (X_test_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}
extra_feats_test = {
    "total_groups": X_test_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_test_df[feature_names] > 0).sum(axis=1),
}
X_test_df = pd.concat([X_test_df, pd.DataFrame({**binary_feats_test, **ratio_feats_test, **extra_feats_test})], axis=1)
print("Shape après feature engineering:", X_test_df.shape)
feats_df, invalids = compute_chem_features(X_test_smiles, smiles_col="SMILES", add_fingerprint=False)
X_test_df = pd.concat([X_test_df, pd.DataFrame({**feats_df})], axis=1)
print("Shape après feature engineering chimique:", X_test_df.shape)

#Génération des Morgan Fingerprints
# --- Application à ton dataset ---
print("🧬 Calcul des empreintes Morgan...")
fps = np.array([morgan_fingerprint(smi, n_bits, radius) for smi in tqdm(X_test_smiles[smiles_col])])
# --- Conversion en DataFrame ---
fp_cols = [f"morgan_{i}" for i in range(n_bits)]
df_fps = pd.DataFrame(fps, columns=fp_cols)
# --- Fusion avec ton DataFrame existant ---
X_test_df = pd.concat([X_test_df.reset_index(drop=True), df_fps], axis=1)
print(f"✅ Ajouté {n_bits} nouvelles colonnes de features Morgan.")
print(f"Nouvelle taille du DataFrame : {X_test_df.shape}")


Shape après feature engineering: (666, 1274)
Shape après feature engineering chimique: (666, 1324)
🧬 Calcul des empreintes Morgan...


100%|██████████| 666/666 [00:00<00:00, 11325.85it/s]

✅ Ajouté 512 nouvelles colonnes de features Morgan.
Nouvelle taille du DataFrame : (666, 1836)


In [10]:
X = X_df
#print(X.head())
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Split Train / Valid
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=42 )


**Import et Selection du modèle**

In [11]:
#from sklearn.model_selection import KFold, cross_val_score
#from sklearn.metrics import mean_absolute_error, make_scorer
#from sklearn.ensemble import StackingRegressor
#from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, cross_val_score
import time
import itertools
#from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor

In [12]:
final_model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.09586251212066181,
    depth=10,
    loss_function = "MAE",
    l2_leaf_reg=10,
    bagging_temperature=2.031454661334689,
    random_state=42,
    verbose=100
    #Ancien paramétre
    #l2_leaf_reg=3
    #depth=10
    #bagging_temperature=2.031454661334689
    #learning_rate=0.09586251212066181
)
final_model.fit(X, y)

0:	learn: 61.7372274	total: 107ms	remaining: 3m 33s
100:	learn: 22.9533588	total: 4.05s	remaining: 1m 16s
200:	learn: 16.1004263	total: 8.06s	remaining: 1m 12s
300:	learn: 12.7793012	total: 12s	remaining: 1m 7s
400:	learn: 10.6324337	total: 16s	remaining: 1m 3s
500:	learn: 9.3079355	total: 19.9s	remaining: 59.4s
600:	learn: 8.2369053	total: 23.8s	remaining: 55.4s
700:	learn: 7.3487891	total: 27.8s	remaining: 51.5s
800:	learn: 6.6320746	total: 31.7s	remaining: 47.5s
900:	learn: 5.9547440	total: 35.6s	remaining: 43.4s
1000:	learn: 5.4573248	total: 39.5s	remaining: 39.4s
1100:	learn: 5.0207952	total: 43.4s	remaining: 35.5s
1200:	learn: 4.6389147	total: 47.4s	remaining: 31.5s
1300:	learn: 4.3333037	total: 51.4s	remaining: 27.6s
1400:	learn: 4.0823023	total: 55.3s	remaining: 23.7s
1500:	learn: 3.8108651	total: 59.3s	remaining: 19.7s
1600:	learn: 3.5948914	total: 1m 3s	remaining: 15.8s
1700:	learn: 3.3743566	total: 1m 7s	remaining: 11.8s
1800:	learn: 3.1385862	total: 1m 11s	remaining: 7.87s


In [13]:
#Test de modèle sur données de test
X_test = X_test_df
predictions = final_model.predict(X_test)
print(predictions)

[364.92964342 299.10911729 193.76672199 207.34431751 247.33329009
 340.05096551 235.92368833 303.95221776 277.35369492 260.10472813
 258.31128593 308.61716456 307.97759486 245.01644995 276.20241809
 586.02710969 276.18377289 302.37458038 402.52125502 159.48468621
 150.94837675 148.49965161 399.53442986 203.73085911 299.0103806
 180.82615064 594.98074819 493.83583823 386.81837669 328.35667425
 381.06568832 326.6294801  284.14230938 189.98951266 328.30696362
 273.54995045 320.62990586 258.65606876 235.23408899 333.2897434
 223.82721109 269.11558946 201.1361772  146.00646812 228.21443813
 306.98211278 332.93403763 208.77320168 288.6937722  350.61349683
 335.96960713 308.49826742 131.47696974 298.82174313 300.63660847
 300.19256948 325.34391006 244.860365   279.91978385 250.1684154
 323.88082485 315.74668102 198.87276999 289.52647076 358.99459923
 190.60140763 278.02896564 319.07260557 249.05719385 320.33744232
 307.00480977 383.86462767 197.27046805 175.54583023 163.79532709
 291.51835584

In [14]:
#Export au format csv
id = test_data_set['id']
Tm = predictions
dict = {"id": id, "Tm": Tm}
submission_df = pd.DataFrame(dict)

# Sauvegarder le DataFrame en fichier CSV
output_csv_path = "submission.csv"
submission_df.to_csv(output_csv_path, index=False, sep=',')

print(f"Fichier CSV généré : {output_csv_path}")
print(submission_df)

Fichier CSV généré : submission.csv
       id          Tm
0    1022  364.929643
1    1146  299.109117
2      79  193.766722
3    2279  207.344318
4    1342  247.333290
..    ...         ...
661  2663  275.059257
662   624  279.575062
663  2655  163.744911
664  2089  232.854143
665  1065  277.678426

[666 rows x 2 columns]


In [15]:
from IPython.display import FileLink
FileLink("submission.csv")

/kaggle/working/submission.csv